In [1]:
import sys, os
sys.path.append("..")
from sqlmodel import select, func
from datetime import datetime, timezone, timedelta
import random

from app.db import get_session, Session, engine
from app.models import Contact, Campaign, ContactCampaignLink, Mail
from app.api.mails import crud_mail

In [3]:
session = Session(engine)

In [5]:
%time Mail.day_count(session)

2022-06-10 08:31:44,446 INFO sqlalchemy.engine.Engine SELECT mail.time_send, mail.is_open, mail.id, mail.contact_id 
FROM mail 
WHERE mail.time_send >= %(time_send_1)s
2022-06-10 08:31:44,448 INFO sqlalchemy.engine.Engine [no key 0.00226s] {'time_send_1': datetime.datetime(2022, 6, 9, 6, 31, 44, 444139, tzinfo=datetime.timezone.utc)}
CPU times: user 6.9 ms, sys: 2.63 ms, total: 9.53 ms
Wall time: 12.7 ms


3

In [77]:
len(session.exec(select(Mail).where(Mail.time_send >= (datetime.now(timezone.utc)  - timedelta(days=30)))).all())

2022-06-03 12:04:13,769 INFO sqlalchemy.engine.Engine SELECT mail.time_send, mail.is_open, mail.id, mail.contact_id 
FROM mail 
WHERE mail.time_send >= %(time_send_1)s
2022-06-03 12:04:13,771 INFO sqlalchemy.engine.Engine [no key 0.00189s] {'time_send_1': datetime.datetime(2022, 5, 4, 10, 4, 13, 768035, tzinfo=datetime.timezone.utc)}


2

In [19]:
now = datetime.now(timezone.utc) 
now, now - timedelta(days = 30) 

(datetime.datetime(2022, 6, 3, 9, 46, 46, 832387, tzinfo=datetime.timezone.utc),
 datetime.datetime(2022, 5, 4, 9, 46, 46, 832387, tzinfo=datetime.timezone.utc))

In [44]:
mails = session.exec(select(Mail)).all()
for mail in mails:
    delta_days = random.randint(10, 50)
    print("delta : ", delta_days)
    t_send = datetime.now(timezone.utc)  - timedelta(days = delta_days) 
    mail.time_send = t_send

2022-06-03 11:49:49,865 INFO sqlalchemy.engine.Engine SELECT mail.time_send, mail.is_open, mail.id, mail.contact_id 
FROM mail
2022-06-03 11:49:49,867 INFO sqlalchemy.engine.Engine [no key 0.00242s] {}
delta :  30
delta :  40
delta :  31
delta :  22
delta :  27
delta :  43
delta :  46
delta :  48


In [45]:
session.commit()

2022-06-03 11:50:09,526 INFO sqlalchemy.engine.Engine UPDATE mail SET time_send=%(time_send)s WHERE mail.id = %(mail_id)s
2022-06-03 11:50:09,529 INFO sqlalchemy.engine.Engine [generated in 0.00336s] ({'time_send': datetime.datetime(2022, 5, 4, 9, 49, 49, 871067, tzinfo=datetime.timezone.utc), 'mail_id': 2}, {'time_send': datetime.datetime(2022, 4, 24, 9, 49, 49, 871276, tzinfo=datetime.timezone.utc), 'mail_id': 3}, {'time_send': datetime.datetime(2022, 5, 3, 9, 49, 49, 871388, tzinfo=datetime.timezone.utc), 'mail_id': 4}, {'time_send': datetime.datetime(2022, 5, 12, 9, 49, 49, 871498, tzinfo=datetime.timezone.utc), 'mail_id': 5}, {'time_send': datetime.datetime(2022, 5, 7, 9, 49, 49, 871605, tzinfo=datetime.timezone.utc), 'mail_id': 6}, {'time_send': datetime.datetime(2022, 4, 21, 9, 49, 49, 871711, tzinfo=datetime.timezone.utc), 'mail_id': 7}, {'time_send': datetime.datetime(2022, 4, 18, 9, 49, 49, 871817, tzinfo=datetime.timezone.utc), 'mail_id': 8}, {'time_send': datetime.datetime(

In [11]:
for i in range(6):
    crud_mail.create(session=session, obj={"contact_id": 1})

2022-06-03 11:42:41,851 INFO sqlalchemy.engine.Engine INSERT INTO mail (time_send, is_open, contact_id) VALUES (%(time_send)s, %(is_open)s, %(contact_id)s) RETURNING mail.id
2022-06-03 11:42:41,853 INFO sqlalchemy.engine.Engine [cached since 25.74s ago] {'time_send': None, 'is_open': False, 'contact_id': None}
2022-06-03 11:42:41,856 INFO sqlalchemy.engine.Engine COMMIT
2022-06-03 11:42:41,859 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:42:41,860 INFO sqlalchemy.engine.Engine SELECT mail.time_send, mail.is_open, mail.id, mail.contact_id 
FROM mail 
WHERE mail.id = %(pk_1)s
2022-06-03 11:42:41,861 INFO sqlalchemy.engine.Engine [cached since 25.73s ago] {'pk_1': 4}
2022-06-03 11:42:41,864 INFO sqlalchemy.engine.Engine INSERT INTO mail (time_send, is_open, contact_id) VALUES (%(time_send)s, %(is_open)s, %(contact_id)s) RETURNING mail.id
2022-06-03 11:42:41,865 INFO sqlalchemy.engine.Engine [cached since 25.75s ago] {'time_send': None, 'is_open': False, 'contact_id': None}

In [4]:
contacts = session.exec(select(Contact)).all()
contacts

2022-06-03 11:38:55,894 INFO sqlalchemy.engine.Engine SELECT contact.meta, contact.email, contact.id 
FROM contact
2022-06-03 11:38:55,896 INFO sqlalchemy.engine.Engine [no key 0.00185s] {}


[Contact(email='louis@genario.co', id=1, meta={'age': 27})]

In [10]:
contacts[0].campaigns[0]

Campaign(id=1, name='first campaign', contacts=[Contact(meta={}, email='louismanhes@icloud.com', id=1, campaigns=[Campaign(id=1, name='first campaign', contacts=[...])]), Contact(meta={}, email='louis@genario.co', id=2)])

In [2]:
session = Session(engine)

In [3]:
campaigns = session.exec(select(Campaign)).all()
campaigns

2022-06-03 11:38:49,781 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-06-03 11:38:49,783 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-06-03 11:38:49,786 INFO sqlalchemy.engine.Engine select current_schema()
2022-06-03 11:38:49,790 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-06-03 11:38:49,792 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-06-03 11:38:49,792 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-06-03 11:38:49,795 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:38:49,811 INFO sqlalchemy.engine.Engine SELECT campaign.name, campaign.id 
FROM campaign
2022-06-03 11:38:49,812 INFO sqlalchemy.engine.Engine [no key 0.00110s] {}


/Users/louismanhes/anaconda3/envs/tinymail/lib/python3.10/site-packages/sqlmodel/orm/session.py:60: SAWarning: Class SelectOfScalar will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = super().execute(


[]

In [5]:
campaigns[0].contacts

[Contact(meta={}, email='louismanhes@icloud.com', id=1),
 Contact(meta={}, email='louis@genario.co', id=2)]

In [30]:
contact_campaign_link = session.exec(select(ContactCampaignLink)).all()
contact_campaign_link

2022-06-01 11:52:05,222 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-01 11:52:05,225 INFO sqlalchemy.engine.Engine SELECT contactcampaignlink.campaign_id, contactcampaignlink.contact_id 
FROM contactcampaignlink
2022-06-01 11:52:05,226 INFO sqlalchemy.engine.Engine [no key 0.00119s] {}


[ContactCampaignLink(campaign_id=1, contact_id=1)]

In [29]:
ok = ContactCampaignLink(campaign_id=1, contact_id=1)
session.add(ok)
session.commit()

2022-06-01 11:52:01,629 INFO sqlalchemy.engine.Engine INSERT INTO contactcampaignlink (campaign_id, contact_id) VALUES (%(campaign_id)s, %(contact_id)s)
2022-06-01 11:52:01,631 INFO sqlalchemy.engine.Engine [generated in 0.00228s] {'campaign_id': 1, 'contact_id': 1}
2022-06-01 11:52:01,641 INFO sqlalchemy.engine.Engine COMMIT
